In [4]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from pydub import AudioSegment
import json
import math
from datetime import timedelta

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("="*60)
print("🎙️  WHISPER TRANSCRIPTION LAB 1.07")
print("="*60)
print("All libraries imported successfully!")

os.makedirs("transcriptions", exist_ok=True)
print("Folders ready!")

🎙️  WHISPER TRANSCRIPTION LAB 1.07
All libraries imported successfully!
Folders ready!


In [5]:

print("\n" + "="*60)
print(" CHECKING AUDIO FOLDER")
print("="*60)

# List all files in audio folder
audio_folder = "audio"
if os.path.exists(audio_folder):
    all_files = os.listdir(audio_folder)
    
    # Filter for audio files
    audio_extensions = ['.mp3', '.wav', '.m4a', '.ogg', '.flac']
    audio_files = [f for f in all_files 
                   if any(f.lower().endswith(ext) for ext in audio_extensions)]
    
    if audio_files:
        print(f"\n Found {len(audio_files)} audio file(s):\n")
        for i, filename in enumerate(audio_files, 1):
            filepath = os.path.join(audio_folder, filename)
            size_mb = os.path.getsize(filepath) / (1024 * 1024)
            print(f"   {i}. {filename}")
            print(f"      Size: {size_mb:.2f} MB")
            print(f"      Path: {filepath}\n")
        
        # Select the first one 
        selected_audio = audio_files[0]
        audio_path = os.path.join(audio_folder, selected_audio)
        
        print(f"Will use: {selected_audio}")
        print(f"File size: {os.path.getsize(audio_path) / (1024*1024):.2f} MB")
        
    else:
        print("No audio files found!")
        print(f"   Please add an audio file (.mp3, .wav, .m4a) to '{audio_folder}/' folder")
else:
    print(f"Folder '{audio_folder}/' doesn't exist!")
    print("   Creating it now...")
    os.makedirs(audio_folder)
    print(f"   Created '{audio_folder}/' folder")
    print("   Please add your audio file to this folder")


 CHECKING AUDIO FOLDER

 Found 1 audio file(s):

   1. meeting_sample.mp3
      Size: 0.89 MB
      Path: audio/meeting_sample.mp3

Will use: meeting_sample.mp3
File size: 0.89 MB


In [11]:
"""
STEP 3: Basic Transcription

"""

print("\n" + "="*60)
print("STEP 3: BASIC TRANSCRIPTION (No Prompts)")
print("="*60)

def basic_transcription(audio_file_path):
    """Simple transcription without any context"""
    print(f"\n Transcribing: {audio_file_path}")
    
    with open(audio_file_path, "rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )
    
    print(" Transcription complete!\n")
    return transcript.text

# Transcribe audio
unprompted_text = basic_transcription(audio_path)

# Display result
print(" TRANSCRIPTION (first 500 characters):")
print("-"*60)
print(unprompted_text[:500])
if len(unprompted_text) > 500:
    print(f"\n... (truncated, full text is {len(unprompted_text)} characters)")

# Save to file
output_file = "transcriptions/unprompted_transcription.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write("="*60 + "\n")
    f.write("UNPROMPTED TRANSCRIPTION\n")
    f.write("(No context provided to Whisper)\n")
    f.write("="*60 + "\n\n")
    f.write(unprompted_text)

print(f"\n Saved to: {output_file}")


STEP 3: BASIC TRANSCRIPTION (No Prompts)

 Transcribing: audio/meeting_sample.mp3
 Transcription complete!

 TRANSCRIPTION (first 500 characters):
------------------------------------------------------------
It was rather interesting just to watch them gathering their materials and bouncing around their, what they call it, kangaroo walk or something like that. Who named it that? I don't know. I bet those men are going to get quite a reception when they get back to Maine. Oh, yes. I'll be so glad when they land back now. But I think that's pretty well a fact, because they've landed so many safely now that I feel relieved. Just getting off of the moon was the thing that was. Have they met with the one

... (truncated, full text is 1095 characters)

 Saved to: transcriptions/unprompted_transcription.txt


In [12]:
"""
STEP 4: Transcription with Prompts (Guided Approach)
"""

print("\n" + "="*60)
print("STEP 4: PROMPTED TRANSCRIPTION")
print("="*60)

# Create prompts based on audio content

prompt_apollo = """
Apollo moon landing conversation circa 1969-1972.
Speakers: fieldworker, informant, fieldworker's sister.
Technical terms: rendezvous, kangaroo walk, 
lunar module, command module, moon landing, Earth return, 
rock samples, soil samples, orbit, circling the moon.
Conversational speech with hesitations, background noise, laughter, overlapping dialogue.
"""

selected_prompt = prompt_apollo  

print(f" Using prompt:")
print("-"*60)
print(selected_prompt)
print("-"*60)

def transcription_with_prompt(audio_file_path, prompt_text):
    """Transcribe with context guidance"""
    print(f"\n Transcribing WITH prompt...")
    
    with open(audio_file_path, "rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            prompt=prompt_text
        )
    
    print(" Prompted transcription complete!\n")
    return transcript.text

# Transcribe with prompt
prompted_text = transcription_with_prompt(audio_path, selected_prompt)

# Display result
print(" PROMPTED TRANSCRIPTION (first 500 characters):")
print("-"*60)
print(prompted_text[:500])
if len(prompted_text) > 500:
    print(f"\n... (truncated, full text is {len(prompted_text)} characters)")

# Save to file
output_file = "transcriptions/prompted_transcription.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write("="*60 + "\n")
    f.write("PROMPTED TRANSCRIPTION\n")
    f.write("="*60 + "\n\n")
    f.write("PROMPT USED:\n")
    f.write("-"*60 + "\n")
    f.write(selected_prompt)
    f.write("\n" + "-"*60 + "\n\n")
    f.write("TRANSCRIPTION:\n")
    f.write("-"*60 + "\n")
    f.write(prompted_text)

print(f"\n Saved to: {output_file}")


STEP 4: PROMPTED TRANSCRIPTION
 Using prompt:
------------------------------------------------------------

Apollo moon landing conversation circa 1969-1972.
Speakers: fieldworker, informant, fieldworker's sister.
Technical terms: rendezvous, kangaroo walk, 
lunar module, command module, moon landing, Earth return, 
rock samples, soil samples, orbit, circling the moon.
Conversational speech with hesitations, background noise, laughter, overlapping dialogue.

------------------------------------------------------------

 Transcribing WITH prompt...
 Prompted transcription complete!

 PROMPTED TRANSCRIPTION (first 500 characters):
------------------------------------------------------------
It was rather interesting just to watch them gathering their materials and bouncing around. What did they call it? Kangaroo walk? Something like that. I don't know. I bet those men are going to get quite a reception when they get back to Earth. Oh yes, I'll be so glad when they land back now. I think

In [9]:
"""
STEP 5: Compare Prompted vs Unprompted
"""

print("\n" + "="*60)
print("STEP 5: COMPARISON")
print("="*60)

print("\n Statistics:")
print(f"   Unprompted length: {len(unprompted_text)} characters")
print(f"   Prompted length:   {len(prompted_text)} characters")
print(f"   Difference:        {abs(len(prompted_text) - len(unprompted_text))} characters")

print("\n First 300 characters comparison:")
print("\n" + "-"*60)
print("WITHOUT PROMPT:")
print("-"*60)
print(unprompted_text[:300])

print("\n" + "-"*60)
print("WITH PROMPT:")
print("-"*60)
print(prompted_text[:300])

print("\n" + "="*60)
print(" OBSERVATIONS:")
print("="*60)
print("""
Look for differences in:
- Technical terms (e.g., "rendezvous", "kangaroo walk")
- Proper nouns and names
- Punctuation and formatting
- Overall clarity and accuracy
""")

# Save comparison
with open("transcriptions/comparison.txt", "w", encoding="utf-8") as f:
    f.write("="*60 + "\n")
    f.write("COMPARISON: PROMPTED VS UNPROMPTED\n")
    f.write("="*60 + "\n\n")
    f.write(f"Unprompted length: {len(unprompted_text)} characters\n")
    f.write(f"Prompted length:   {len(prompted_text)} characters\n\n")
    f.write("-"*60 + "\n")
    f.write("UNPROMPTED:\n")
    f.write("-"*60 + "\n")
    f.write(unprompted_text)
    f.write("\n\n" + "="*60 + "\n\n")
    f.write("-"*60 + "\n")
    f.write("PROMPTED:\n")
    f.write("-"*60 + "\n")
    f.write(prompted_text)

print("\n Comparison saved to: transcriptions/comparison.txt")


STEP 5: COMPARISON

 Statistics:
   Unprompted length: 1095 characters
   Prompted length:   1055 characters
   Difference:        40 characters

 First 300 characters comparison:

------------------------------------------------------------
WITHOUT PROMPT:
------------------------------------------------------------
It was rather interesting just to watch them gathering their materials and bouncing around their, what they call it, kangaroo walk or something like that. Who named it that? I don't know. I bet those men are going to get quite a reception when they get back to Maine. Oh, yes. I'll be so glad when th

------------------------------------------------------------
WITH PROMPT:
------------------------------------------------------------
It was rather interesting just to watch them gathering their materials and bouncing around. What did they call it? Kangaroo walk? Something like that. I don't know. I bet those men are going to get quite a reception when they get back to Earth

In [16]:
"""
STEP 6: Implementing Audio Chunking
"""

print("\n" + "="*60)
print("STEP 6: AUDIO CHUNKING")
print("="*60)

# Check file size
file_size_mb = os.path.getsize(audio_path) / (1024 * 1024)
print(f"\n Audio file size: {file_size_mb:.2f} MB")

if file_size_mb > 25:
    print("  File is larger than 25MB - chunking required!")
    needs_chunking = True
else:
    print(" File is under 25MB - chunking optional")
    needs_chunking = False

def chunk_audio(audio_file_path, chunk_length_minutes=5):
    """
    Split audio into chunks
    
    Args:
        audio_file_path: Path to audio file
        chunk_length_minutes: Length of each chunk in minutes
    
    Returns:
        List of chunk file paths
    """
    print(f"\n Chunking audio into {chunk_length_minutes}-minute segments...")
    
    # Load audio
    audio = AudioSegment.from_file(audio_file_path)
    
    # Calculate chunk parameters
    chunk_length_ms = chunk_length_minutes * 60 * 1000
    total_length_ms = len(audio)
    total_minutes = total_length_ms / 1000 / 60
    num_chunks = math.ceil(total_length_ms / chunk_length_ms)
    
    print(f"   Total duration: {total_minutes:.2f} minutes")
    print(f"   Creating {num_chunks} chunks\n")
    
    chunk_paths = []
    
    for i in range(num_chunks):
        start_ms = i * chunk_length_ms
        end_ms = min((i + 1) * chunk_length_ms, total_length_ms)
        
        # Extract chunk
        chunk = audio[start_ms:end_ms]
        
        # Save chunk
        chunk_filename = f"audio/chunk_{i:03d}.mp3"
        chunk.export(chunk_filename, format="mp3")
        chunk_paths.append(chunk_filename)
        
        chunk_duration = (end_ms - start_ms) / 1000 / 60
        print(f"    Chunk {i+1}/{num_chunks}: {chunk_filename} ({chunk_duration:.2f} min)")
    
    return chunk_paths

# Create chunks
chunks = chunk_audio(audio_path, chunk_length_minutes=5)

print(f"\n Created {len(chunks)} chunks!")
print(f" Chunks saved in 'audio/' folder")


STEP 6: AUDIO CHUNKING

 Audio file size: 0.89 MB
 File is under 25MB - chunking optional

 Chunking audio into 5-minute segments...
   Total duration: 1.44 minutes
   Creating 1 chunks

    Chunk 1/1: audio/chunk_000.mp3 (1.44 min)

 Created 1 chunks!
 Chunks saved in 'audio/' folder


In [19]:
"""
STEP 7: Transcribe Chunks with Timestamps
"""

print("\n" + "="*60)
print("STEP 7: CHUNKED TRANSCRIPTION WITH TIMESTAMPS")
print("="*60)

def transcribe_chunks_with_timestamps(chunk_paths, prompt=None):
    """
    Transcribe multiple chunks and maintain timestamps
    
    Args:
        chunk_paths: List of chunk file paths
        prompt: Optional context prompt
    
    Returns:
        List of segments with timestamps
    """
    print(f"\n Transcribing {len(chunk_paths)} chunks...\n")
    
    all_segments = []
    cumulative_time = 0.0
    
    for i, chunk_path in enumerate(chunk_paths):
        print(f"   Processing chunk {i+1}/{len(chunk_paths)}...", end=" ")
        
        with open(chunk_path, "rb") as audio_file:
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
                response_format="verbose_json",
                timestamp_granularities=["segment"],
                prompt=prompt if prompt else None
            )
        
        # Adjust timestamps for chunk position
        for segment in transcript.segments:
            adjusted_segment = {
                "start": segment.start + cumulative_time,    # Changed from segment['start']
                "end": segment.end + cumulative_time,        # Changed from segment['end']
                "text": segment.text                         # Changed from segment['text']
            }
            all_segments.append(adjusted_segment)
        
        cumulative_time += transcript.duration
        print(f" ({transcript.duration:.2f}s)")
    
    print(f"\n All chunks transcribed! Total segments: {len(all_segments)}")
    return all_segments

# Transcribe all chunks
segments = transcribe_chunks_with_timestamps(chunks, prompt=selected_prompt)

# Display sample
print("\n Sample segments (first 5):")
print("-"*60)
for seg in segments[:5]:
    minutes = int(seg['start'] // 60)
    seconds = int(seg['start'] % 60)
    print(f"[{minutes:02d}:{seconds:02d}] {seg['text']}")


STEP 7: CHUNKED TRANSCRIPTION WITH TIMESTAMPS

 Transcribing 1 chunks...

   Processing chunk 1/1...  (86.19s)

 All chunks transcribed! Total segments: 10

 Sample segments (first 5):
------------------------------------------------------------
[00:00]  It was rather interesting just to watch them gathering their materials and bouncing around their, what do they call it, kangaroo walk or something like that.
[00:13]  Who named it that? I don't know.
[00:15]  I bet those men are going to get quite a reception when they get back.
[00:19]  Yes, I'll be so glad when they land back now, but I think that's pretty well a fact because they've landed so many safely now that I feel relieved.
[00:33]  Just getting off of the moon was the thing that was...


In [20]:
"""
STEP 8: Export in Multiple Formats
Save transcriptions as TXT, JSON, and SRT
"""

print("\n" + "="*60)
print("STEP 8: EXPORTING RESULTS")
print("="*60)

# Export as TEXT
def export_to_text(segments, output_path="transcriptions/transcript.txt"):
    """Export as readable text with timestamps"""
    with open(output_path, "w", encoding="utf-8") as f:
        f.write("MEETING TRANSCRIPTION\n")
        f.write("=" * 60 + "\n")
        f.write(f"Total segments: {len(segments)}\n")
        f.write("=" * 60 + "\n\n")
        
        for seg in segments:
            minutes = int(seg['start'] // 60)
            seconds = int(seg['start'] % 60)
            f.write(f"[{minutes:02d}:{seconds:02d}] {seg['text'].strip()}\n")
    
    print(f" Text: {output_path}")

# Export as JSON
def export_to_json(segments, output_path="transcriptions/transcript.json"):
    """Export as JSON"""
    data = {
        "transcription": segments,
        "total_segments": len(segments),
        "duration_seconds": segments[-1]['end'] if segments else 0
    }
    
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    
    print(f" JSON: {output_path}")

# Export as SRT (subtitles)
def export_to_srt(segments, output_path="transcriptions/transcript.srt"):
    """Export as SRT subtitle format"""
    with open(output_path, "w", encoding="utf-8") as f:
        for i, seg in enumerate(segments, start=1):
            start_time = str(timedelta(seconds=seg['start'])).split('.')[0] + ',000'
            end_time = str(timedelta(seconds=seg['end'])).split('.')[0] + ',000'
            
            if len(start_time.split(':')[0]) == 1:
                start_time = '0' + start_time
            if len(end_time.split(':')[0]) == 1:
                end_time = '0' + end_time
            
            f.write(f"{i}\n")
            f.write(f"{start_time} --> {end_time}\n")
            f.write(f"{seg['text'].strip()}\n\n")
    
    print(f" SRT:  {output_path}")

# Export in all formats
export_to_text(segments)
export_to_json(segments)
export_to_srt(segments)

print("\n All files saved in 'transcriptions/' folder!")


STEP 8: EXPORTING RESULTS
 Text: transcriptions/transcript.txt
 JSON: transcriptions/transcript.json
 SRT:  transcriptions/transcript.srt

 All files saved in 'transcriptions/' folder!
